In [61]:
import numpy as np
import pandas as pd

carData = pd.read_csv('motor_vehicles.csv')
print(carData.shape)  # gives a tuple with the shape of data frame

(1728, 7)


In [62]:
print(carData.head())

  fuel_efficiency maintenance doors    type   size safety RATING
0       excellent       vhigh   two  sports  small    low   JUNK
1       excellent       vhigh   two  sports  small    med   JUNK
2       excellent       vhigh   two  sports  small   high   JUNK
3       excellent       vhigh   two  sports    med    low   JUNK
4       excellent       vhigh   two  sports    med    med   JUNK


In [63]:
splitRatio = .85

In [64]:
price_labels = ['low', 'medium', 'high', 'luxury']
maint_labels = ['low_cost', 'med_cost', 'high_cost', 'very_high_cost']
trunk_labels = ['small', 'med', 'big']
safety_labels = ['poor', 'ave', 'good']

In [65]:
# Do the shuffle!
reCarData = carData.reindex(np.random.permutation(carData.index))
shuffCarData = reCarData.reset_index()

In [66]:
shuffCarData

,index,fuel_efficiency,maintenance,doors,type,size,safety,RATING
0,368,excellent,low,three,sedan,big,high,OKAY
1,1143,med,med,four,sedan,small,low,JUNK
2,283,excellent,med,four,sedan,med,med,OKAY
3,628,good,high,many,sports,big,med,JUNK
4,455,good,vhigh,two,wagon,med,high,JUNK
...,...,...,...,...,...,...,...,...
1723,1210,med,low,two,wagon,med,med,OKAY
1724,816,good,low,four,sports,big,low,JUNK
1725,869,med,vhigh,two,sports,med,high,JUNK
1726,459,good,vhigh,three,sports,small,low,JUNK


In [67]:
shuffCarData.pop('index')

0        368
1       1143
2        283
3        628
4        455
        ... 
1723    1210
1724     816
1725     869
1726     459
1727    1269
Name: index, Length: 1728, dtype: int64

In [68]:
shuffCarData

,fuel_efficiency,maintenance,doors,type,size,safety,RATING
0,excellent,low,three,sedan,big,high,OKAY
1,med,med,four,sedan,small,low,JUNK
2,excellent,med,four,sedan,med,med,OKAY
3,good,high,many,sports,big,med,JUNK
4,good,vhigh,two,wagon,med,high,JUNK
...,...,...,...,...,...,...,...
1723,med,low,two,wagon,med,med,OKAY
1724,good,low,four,sports,big,low,JUNK
1725,med,vhigh,two,sports,med,high,JUNK
1726,good,vhigh,three,sports,small,low,JUNK


In [69]:
# function definition: getting the counts (frequencies of occurrence)
def count(data,colname,label,target):    # for whichever data set, pick a column and one value in that column
                                         # and an output label (target) to compare against
    condition = (data[colname] == label) & (data['RATING'] == target)
    print(data[condition])
    return len(data[condition])

In [70]:
# list of predicted values
predicted = []

# dictionary to store probabilities
probabilities = {0:{},1:{}}

#length of current training set
train_len = int(splitRatio * len(shuffCarData))
print(train_len)

#splitting the data into training and test sets
training = shuffCarData.iloc[:train_len,:] # the first train_len rows, all columns

1468


In [71]:
# debugging step: looking at the training set
training

,fuel_efficiency,maintenance,doors,type,size,safety,RATING
0,excellent,low,three,sedan,big,high,OKAY
1,med,med,four,sedan,small,low,JUNK
2,excellent,med,four,sedan,med,med,OKAY
3,good,high,many,sports,big,med,JUNK
4,good,vhigh,two,wagon,med,high,JUNK
...,...,...,...,...,...,...,...
1463,crap,vhigh,many,sports,med,med,JUNK
1464,excellent,low,three,wagon,med,high,OKAY
1465,med,vhigh,many,sedan,small,low,JUNK
1466,excellent,med,many,wagon,big,low,JUNK


In [72]:
test_X = shuffCarData.iloc[train_len:,:-1] # features in the test set
test_y = shuffCarData.iloc[train_len:,-1]  # targets (the last entry in each row; i.e., the last column)

In [73]:
# debugging
print(test_X)

     fuel_efficiency maintenance  doors    type   size safety
1468            crap        high    two   wagon    big   high
1469             med        high   many   sedan    big   high
1470            crap         med   many   wagon    med   high
1471            crap         med   four   sedan  small    med
1472             med        high    two   wagon    big   high
...              ...         ...    ...     ...    ...    ...
1723             med         low    two   wagon    med    med
1724            good         low   four  sports    big    low
1725             med       vhigh    two  sports    med   high
1726            good       vhigh  three  sports  small    low
1727             med         low   many  sports  small    low

[260 rows x 6 columns]


In [74]:
# debugging
print(test_y)

1468    DANK
1469    OKAY
1470    DANK
1471    OKAY
1472    OKAY
        ... 
1723    OKAY
1724    JUNK
1725    JUNK
1726    JUNK
1727    JUNK
Name: RATING, Length: 260, dtype: object


In [75]:
# checking for dimension match
print(test_X.shape)
print(test_y.shape)

(260, 6)
(260,)


In [76]:
training

,fuel_efficiency,maintenance,doors,type,size,safety,RATING
0,excellent,low,three,sedan,big,high,OKAY
1,med,med,four,sedan,small,low,JUNK
2,excellent,med,four,sedan,med,med,OKAY
3,good,high,many,sports,big,med,JUNK
4,good,vhigh,two,wagon,med,high,JUNK
...,...,...,...,...,...,...,...
1463,crap,vhigh,many,sports,med,med,JUNK
1464,excellent,low,three,wagon,med,high,OKAY
1465,med,vhigh,many,sedan,small,low,JUNK
1466,excellent,med,many,wagon,big,low,JUNK


In [77]:
# counts of prior 'LOW' and 'HIGH' labels (targets)
count_LOW = count(training, 'RATING', 'JUNK', 'JUNK')
count_HIGH = count(training, 'RATING', 'OKAY', 'OKAY')

prior_LOW = count_LOW / len(training)
prior_HIGH = count_HIGH / len(training)

print('The prior \'LOW\' count is', count_LOW)
print('and \'HIGH\',',count_HIGH, '.')
print('The prior probability of \'LOW\' is', prior_LOW)
print('and of \'HIGH\',', prior_HIGH, '.')

     fuel_efficiency maintenance  doors    type   size safety RATING
1                med         med   four   sedan  small    low   JUNK
3               good        high   many  sports    big    med   JUNK
4               good       vhigh    two   wagon    med   high   JUNK
6               crap         med   four  sports    big    low   JUNK
7                med       vhigh  three   sedan    med    low   JUNK
...              ...         ...    ...     ...    ...    ...    ...
1462            good       vhigh   four   wagon    big    med   JUNK
1463            crap       vhigh   many  sports    med    med   JUNK
1465             med       vhigh   many   sedan  small    low   JUNK
1466       excellent         med   many   wagon    big    low   JUNK
1467       excellent       vhigh   many   sedan    med    med   JUNK

[1037 rows x 7 columns]
     fuel_efficiency maintenance  doors   type   size safety RATING
0          excellent         low  three  sedan    big   high   OKAY
2          

In [78]:
# TRAINING: learning the priors and class-conditionals from the training set
for col in training.columns[:-1]:
    probabilities[0][col] = {}
    probabilities[1][col] = {}

    if col == "price":
        for k in price_labels:
            count_k_LOW = count(training, col, k, 'LOW')
            count_k_HIGH = count(training, col, k, 'HIGH')
            probabilities[0][col][k] = count_k_LOW / count_LOW
            probabilities[1][col][k] = count_k_HIGH / count_HIGH
            probabilities[0][col][k] += 0.001 # throwing in a quick-and-dirty approximation of the
            probabilities[1][col][k] += 0.001 # "m estimate" for avoiding zeros in the product    
    else:
        if col == "maint_cost":
            for k in maint_labels:
                count_k_LOW = count(training, col, k, 'LOW')
                count_k_HIGH = count(training, col, k, 'HIGH')
                probabilities[0][col][k] = count_k_LOW / count_LOW
                probabilities[1][col][k] = count_k_HIGH / count_HIGH
                probabilities[0][col][k] += 0.001 # throwing in a quick-and-dirty approximation of the
                probabilities[1][col][k] += 0.001 # "m estimate" for avoiding zeros in the product
        else:
            if col == "trunk":
                for k in trunk_labels:
                    count_k_LOW = count(training, col, k, 'LOW')
                    count_k_HIGH = count(training, col, k, 'HIGH')
                    probabilities[0][col][k] = count_k_LOW / count_LOW
                    probabilities[1][col][k] = count_k_HIGH / count_HIGH
                    probabilities[0][col][k] += 0.001 # throwing in a quick-and-dirty approximation of the
                    probabilities[1][col][k] += 0.001 # "m estimate" for avoiding zeros in the product
            else:
                for k in safety_labels:
                    count_k_LOW = count(training, col, k, 'LOW')
                    count_k_HIGH = count(training, col, k, 'HIGH')
                    probabilities[0][col][k] = count_k_LOW / count_LOW
                    probabilities[1][col][k] = count_k_HIGH / count_HIGH
                    probabilities[0][col][k] += 0.001 # throwing in a quick-and-dirty approximation of the
                    probabilities[1][col][k] += 0.001 # "m estimate" for avoiding zeros in the product

Empty DataFrame
Columns: [fuel_efficiency, maintenance, doors, type, size, safety, RATING]
Index: []
Empty DataFrame
Columns: [fuel_efficiency, maintenance, doors, type, size, safety, RATING]
Index: []
Empty DataFrame
Columns: [fuel_efficiency, maintenance, doors, type, size, safety, RATING]
Index: []
Empty DataFrame
Columns: [fuel_efficiency, maintenance, doors, type, size, safety, RATING]
Index: []
Empty DataFrame
Columns: [fuel_efficiency, maintenance, doors, type, size, safety, RATING]
Index: []
Empty DataFrame
Columns: [fuel_efficiency, maintenance, doors, type, size, safety, RATING]
Index: []
Empty DataFrame
Columns: [fuel_efficiency, maintenance, doors, type, size, safety, RATING]
Index: []
Empty DataFrame
Columns: [fuel_efficiency, maintenance, doors, type, size, safety, RATING]
Index: []
Empty DataFrame
Columns: [fuel_efficiency, maintenance, doors, type, size, safety, RATING]
Index: []
Empty DataFrame
Columns: [fuel_efficiency, maintenance, doors, type, size, safety, RATING]


In [79]:
import pprint

# Sample dictionary for demonstration
pp = pprint.PrettyPrinter(indent=4)

# Format each value in the nested dictionaries and print
formatted_probabilities = {
    key: {inner_key: {sub_key: "{:.3f}".format(sub_value) for sub_key, sub_value in inner_value.items()}
           for inner_key, inner_value in value.items()}
    for key, value in probabilities.items()
}

pp.pprint(formatted_probabilities)

{   0: {   'doors': {'ave': '0.001', 'good': '0.001', 'poor': '0.001'},
           'fuel_efficiency': {   'ave': '0.001',
                                  'good': '0.001',
                                  'poor': '0.001'},
           'maintenance': {'ave': '0.001', 'good': '0.001', 'poor': '0.001'},
           'safety': {'ave': '0.001', 'good': '0.001', 'poor': '0.001'},
           'size': {'ave': '0.001', 'good': '0.001', 'poor': '0.001'},
           'type': {'ave': '0.001', 'good': '0.001', 'poor': '0.001'}},
    1: {   'doors': {'ave': '0.001', 'good': '0.001', 'poor': '0.001'},
           'fuel_efficiency': {   'ave': '0.001',
                                  'good': '0.001',
                                  'poor': '0.001'},
           'maintenance': {'ave': '0.001', 'good': '0.001', 'poor': '0.001'},
           'safety': {'ave': '0.001', 'good': '0.001', 'poor': '0.001'},
           'size': {'ave': '0.001', 'good': '0.001', 'poor': '0.001'},
           'type': {'ave': '0.001'

In [80]:
# TESTING: reading rows from the test set, checking the value of each feature and comparing with the target outcome
for row in range(len(test_X)):
    prod_LOW = prior_LOW
    prod_HIGH = prior_HIGH
        
    for feature in test_X.columns:
        # Check if the feature value exists in the probabilities dictionary:
        if test_X[feature].iloc[row] in probabilities[0][feature]:
            prod_LOW *= probabilities[0][feature][test_X[feature].iloc[row]]
        else:
            prod_LOW *= 0  # or some default value if the feature value is not found

        if test_X[feature].iloc[row] in probabilities[1][feature]:
            prod_HIGH *= probabilities[1][feature][test_X[feature].iloc[row]]
        else:
            prod_HIGH *= 0  # or some default value if the feature value is not found
            
    # predicting the outcome
    if prod_LOW > prod_HIGH:
        predicted.append('LOW')
    else:
        predicted.append('HIGH')

# Initialize correct and incorrect counts outside the loop
correct = 0
incorrect = 0

for j in range(len(test_y)):
    if predicted[j] == 'LOW':
        if test_y.iloc[j] == 'LOW':
            correct += 1
        else:
            incorrect += 1
    else:
        if test_y.iloc[j] == 'LOW':
            incorrect += 1  # This should be incorrect since predicted is 'HIGH'
        else:
            correct += 1  # This should be correct since predicted is 'HIGH'

print('The accuracy is', correct / (correct + incorrect))

The accuracy is 1.0
